# Learning Probability Distribution over Sequences

Let's consider 1-dimensional sequences, where items are scalars.

First, estimate the probability distribution over the next item in the sequence:

$ P(x^{\langle t \rangle} | x^{\langle t-1 \rangle}, x^{\langle t-2 \rangle}, \dots, x^{\langle 1 \rangle}) $

Then, the probability of a sequence $\textbf{x}$ is:
$$
P(\textbf{x}) = \prod_{t=1}^{T_x}
{P\big(x^{\langle t \rangle} | x^{\langle t-1 \rangle}, x^{\langle t-2 \rangle}, \dots, x^{\langle 1 \rangle}\big)}
$$

Note that $P(x^{\langle 1 \rangle})$ is not computable, because there is nothing to condition it with. To avoid this inconvenice, training sequences can be prefixed with Start Token that signals sequence start. Corresponding target value for Start Token is the first item in the sequence. In testing, feeding Start Token will output the probability distribution over the first item in every possible sequence. Similarly, to learn the probability of a sequence ending, training sequences can be suffixed with End Token, which will appear as the last target item but never as an input item.

# Categorical Distribution

For example, if the distribution is categorical with K possible outcomes:

$
i = 1, \dots, K
\\
\textbf{x}^{\langle t \rangle} \in \{0, 1\}^{K}
\\
$

Then the output layer can be softmax activation of the hidden state:
$$
\hat{y}^{\langle t \rangle}_i
= P\big(
    x^{\langle t \rangle}_i = 1 |
    \textbf{x}^{\langle t-1 \rangle}, 
    \textbf{x}^{\langle t-2 \rangle}, 
    \dots, 
    \textbf{x}^{\langle 1 \rangle}
\big) 
= \frac
{\text{exp}(h^{\langle t \rangle}_i)}
{\sum_{j=1}^K{\text{exp}(h^{\langle t \rangle}_j)}}
$$

# Generative Inference

It's possible to sample sequences from the learned distribution by feeding the Start Token, randomly choosing an item using the estimated distribution, then feeding this value as the next input item. This process can be repeated until the End Token is sampled. 

# DeepAR

Original paper: Salinas, D., et al. (2019)
https://arxiv.org/abs/1704.04110

DeepAR is trained with several time series simultaneously, so the model learns across multiple related time series. This allows the model to generalize to new but similar time series where little historical data is available (a task called cold start). Additionally, the model is given auxillary data about each target time series in the form of a covariate time series that is also required to known during the prediction window. Each time series is converted into multiple sequences by randomly choosing the start time. The model's goal is to estimate the conditional probability distribution:

$$
P\big(
\textbf{x}_i^{\langle t_0:T \rangle} | 
\textbf{x}_i^{\langle 1:t-1 \rangle}, 
\textbf{z}_i^{\langle 1:T \rangle}
\big)
$$

Sequence $\textbf{x}_i$ is the i-th target sequence, while $\textbf{z}$ is the i-th covariate sequence. 

During inference, conditioning window $[1, t_0 - 1]$ is the time period before the forecast is made, 
while prediction window $[t_0, T]$ is the time period that needs to be forecasted. 
Therefore, $\textbf{x}$ is known until and including $t_0 - 1$, while covariate $\textbf{z}$ is known for the full window $[1, T]$. Each sequence pair $(\textbf{x}_i, \textbf{z}_i)$ may cover a different constant-duration period of universal time.

Similarly to the example in the previous section, the joint distribution is constructed from chained conditionals:
$$
P\big(
    \textbf{x}_i^{\langle t_0:T \rangle} | 
    \textbf{x}_i^{\langle 1:t-1 \rangle}, 
    \textbf{z}_i^{\langle 1:T \rangle}
\big) 
= \prod_{t=1}^{T}
{P\big(
    x_i^{\langle t \rangle} | x_i^{\langle 1:t-1 \rangle}, \textbf{z}_i^{\langle 1:T \rangle}
\big)}
$$

The model does not learn the distribution directly.
Instead the model generates distribution parameters, which are fed into the distribution's PDF/PMF.
The choice of the PDF/PMF is driven by the statistical properties of the target sequence's items:
* for categorical sequences, the categorical distribution can be modeled with softmax activation
* for real-valued sequences, the Gaussian distribution can be learned via mean and standard deviation
* for non-negative integer sequences, the Negative Binomial distribution can be learned via mean and shape

The distribution's PDF/PMF have to be differentiable, so that the cost function's gradient can be computed, and the model can be trained with gradient descent, since the RNN and the distribution parameters need to be learned together.

The cost function is the sum of negative log-likelihood over every time step of every target sequence:

$$
C = 
\sum_{i=1}^N{
    \sum_{t=0}^T{
        - \text{log} \, \ell \big(
            x_i^{\langle t \rangle} \, | \, \theta(\textbf{h}_i^{\langle t \rangle})
        \big)
    }
}
$$

In this definition, $\theta(\textbf{h}_i^{\langle t \rangle})$ is the function converting the LSTM's hidden state at a particular step of a particular sequence to the probability distribution parameters.
Likelihood $\ell$ is the probability density (for continuous distributions) or the probability mass (for discrete distributions) evaluated at a particular value $x_i^{\langle t \rangle}$ given the distribution's parameters.

The LSTM cell is initialized with 0 vector, and given 0 vector as the ground truth in the first step. In this step, the cost is calculated against the sequence's first item.

In training, the distinction between conditioning and prediction windows is effectively absent due to Teacher Forcing.
Ground truth is repetitively fed into the model with a lag of 1 period, while the cost is calculated against the unlagged sequence item. Ground truth is used over both conditioning and prediction windows. Random samples are used only in testing to estimate summary statistics.

# MQ-RNN

MQ-RNN is trained to predict specific quantiles of the time series. Individual predictions are penalized with quantile loss function:
$$ L_q(y, \hat{y}) = max(0, q(y - \hat{y}) + max(0, (1-q)(\hat{y} - y) $$

$$
L_q(y, \hat{y}) =
\begin{cases}
  q(y - \hat{y}) & \text{if } \hat{y} < y \\    
  (1-q)(\hat{y} - y) & \text{otherwise}
\end{cases}
$$

For the i-th target sequence, the total cost over all time steps $t$, quantiles $q$, and prediction horizons $k$ is:

$$
C_i = 
\sum_t\sum_q\sum_k{
    L_q(y^{\langle t+k \rangle}, \hat{y}_q^{\langle t+k \rangle})
}
$$

Decoder consists of two Multi-layer Perceptrons. The global MLP creates context for $K$ prediction horizons, while the local MLP outputs quantiles for a specific prediction horizon $k$. Context consists of a horizon-specific, and a horizon-agnostic parts. Horizon-specific context carries awareness of the distance between Forecast Creation Point $t$ and the prediction horizon $t+k$.

$$
\begin{align}
\textbf{g}^{\langle t \rangle}
    & = (\textbf{c}^{\langle t+1 \rangle}, \dots, \textbf{c}^{\langle t+K \rangle}, \textbf{c}_a^{\langle t \rangle})  \\
    & = g(\textbf{h}^{\langle t \rangle}, \textbf{x}_f^{\langle t: \rangle})
\end{align}
$$

$$
\begin{align}
\hat{\textbf{y}}^{\langle t+k \rangle} 
    & = (\hat{y}_{1}^{\langle t+k \rangle}, \dots, \hat{y}_{Q}^{\langle t+k \rangle}) \\
    & = f(\textbf{c}^{\langle t+k \rangle}, \textbf{c}_a^{\langle t \rangle}, x_f^{\langle t+k \rangle})
\end{align}
$$